In [29]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from prettytable import PrettyTable
import re
import warnings
warnings.filterwarnings("ignore")

In [30]:
train_data = pd.read_csv('data/multilabelClassification/train.csv')
test_data = pd.read_csv('data/multilabelClassification/test.csv')
dev_data = pd.read_csv('data/multilabelClassification/dev.csv')

In [31]:
train_data.head()

,Description,Commenting,Ogling/Facial Expressions/Staring,Touching /Groping
0,"Was walking along crowded street, holding mums...",0,0,1
1,This incident took place in the evening.I was ...,0,1,0
2,I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...,1,0,0
3,Incident happened inside the train,0,0,0
4,I witnessed an incident when a chain was bruta...,0,0,0


In [32]:
tag_combination = {(1,1,1):0,(1,1,0):0,(1,0,1):0,(1,0,0):0,(0,1,1):0,(0,1,0):0,(0,0,1):0,(0,0,0):0}

for n in range(len(train_data)):
    c = train_data.iloc[n]['Commenting']
    o = train_data.iloc[n]['Ogling/Facial Expressions/Staring']
    g = train_data.iloc[n]['Touching /Groping']
    tag_combination[(c,o,g)]+=1

In [33]:
pt = PrettyTable()

pt.field_names = [train_data.columns[1],train_data.columns[2],train_data.columns[3],"Count"]
for comb in tag_combination:
    pt.add_row([comb[0],comb[1],comb[2],tag_combination[comb]])
print(pt)

+------------+-----------------------------------+-------------------+-------+
| Commenting | Ogling/Facial Expressions/Staring | Touching /Groping | Count |
+------------+-----------------------------------+-------------------+-------+
|     1      |                 1                 |         1         |  243  |
|     1      |                 1                 |         0         |  600  |
|     1      |                 0                 |         1         |  333  |
|     1      |                 0                 |         0         |  1644 |
|     0      |                 1                 |         1         |  145  |
|     0      |                 1                 |         0         |  538  |
|     0      |                 0                 |         1         |  1445 |
|     0      |                 0                 |         0         |  2253 |
+------------+-----------------------------------+-------------------+-------+


In [34]:
row_with_html_tag=0
for row in tqdm(range(len(train_data['Description']))):
    searchObject = re.search(r'<.*?>',train_data['Description'][row])
    if searchObject!=None:
        row_with_html_tag+=1
        train_data['Description'][row]=re.sub(r'<.*?>',"",train_data['Description'][row])   

if row_with_html_tag==0:
    print("No comment with html tags")
else:
    print("Number of rows with html tags -> {0}".format(row_with_html_tag))

100%|██████████| 7201/7201 [00:00<00:00, 41676.35it/s]

No comment with html tags


In [35]:
row_with_numbers_special_char=0
for row in tqdm(range(len(train_data['Description']))):
    searchObject = re.search(r'[^a-zA-Z ]',train_data['Description'][row])
    if searchObject!=None:
        row_with_numbers_special_char+=1
        train_data['Description'][row] = re.sub(r'[^a-zA-Z ]',"",train_data['Description'][row])
        
if row_with_numbers_special_char==0:
    print("No comment with numbers and special characters")
else:
    print("Number of rows numbers and special characters -> {0}".format(row_with_numbers_special_char))

100%|██████████| 7201/7201 [00:00<00:00, 7670.54it/s]

Number of rows numbers and special characters -> 5214
